# <center> **Лекция 5:** Компьютерное зрение. Задача Object Detection. YOLO, YOLOv2, YOLO9000, YOLOv3/v4. </center>
----
<br>
</br>

<center> <img src=./../src/imgs/yolo_img.png> </center>

<br>
</br>

## __План на сегодня:__

----
### 1. _YOLO и развитие YOLO. Структура и архитектура YOLOv2, YOLOv3/v4 детекторов._
### 2. _Darknet-19 и Darknet-53. Репозиторий ultralytics._

# Часть 1:

____

# YOLO (You Only Look Once):

----


<br>
</br>

<center> <img src=./../src/imgs/yolo_detection.png> </center>

<br>
</br>

"Новый" подход к решению задачи Object Detection - вместо того, чтобы использовать классификаторы для задач обнаружения объектов (как в DPM, R-CNN, Fast R-CNN etc.) мы можем определить задачу регрессии к боксам (bounding boxes) и связанным с ними вероятностями классов (class probabilities).

Сама модель YOLO состоит из одной CNN, которая делает предсказания за один проход и может быть использована и обучена в варианте end-to-end. Базовая модель может работать real-time на __скорости 45 fps__ более улучшенные версии достигают 155 fps и выше обходя другие SOTA детекторы по метрике __mAP__. Но есть нюанс, YOLO менее точен в локализации объектов, однако выдает меньше ложных срабатываний (FP, false positives).


----

### Unified Detection:

----

<br>
</br>

<center> <img src=./../src/imgs/unified_detection.png> </center>

<br>
</br>


Какова общая идея:

Объединяем все компоненты детектора в единую сеть, что позволит нам:

1. Обучить модель end-to-end.
2. Делать предсказания за один проход и работать real-time.
3. Использовать контекстную информацию со всего изображения для предсказания боксов.

<br>
</br>

----

### Grid Cell:

----

<br>
</br>

<center> <img src=./../src/imgs/grid_cell.png> </center>

<br>
</br>

YOLO разбивает входящее изображение на сетку размером __MxM__. Если центр объекта попадает в одну из ячеек сетки, то данная ячейка будет ответственна за детекцию этого объекта.

Каждая ячейка предсказывает фиксированное число B боксов (bounding boxes) и оценок (box confidence scores) для них. При это независимо от числа B в каждой ячейке может детектиться одновременно только и только один объект. Кроме того, каждая ячейка предсказывает C условных вероятностей для класса объекта (conditional class probabilities). Идея всего этого механизма показана на скриншоте:

<br>
</br>

<center> <img src=./../src/imgs/yolodetections.png> </center>

<br>
</br>


Каждый такой бокс состоит из пятерки координат __[x, y, w, h, confidence_score]__:



* Координаты (x,y) - центр бокса относительно границ ячейки (т.е. показываю смещение)
* Координаты (w, h) - нормализованы относительно всего изображения.
* Все координаты находятся в диапазоне от [0, 1].
* _Box Confidence score_ отражает уверенности модели в том, что бокс содержит объект и как точно этот объект локализован.
* _Conditional Class Probabilities_ описывает вероятности того, к какому классу принадлежит объект (для каждой клетки одна вероятность на каждый класс независимо от числа боксов B).
* _Class Confidence Score_ вычисляется на этапе теста для каждого бокса и дает итоговую оценку отражающую точность классификации объекта и точность локализации объекта.



Как устроен весь процесс:


<br>
</br>

<center> <img src=./../src/imgs/box_confidence.png> </center>



<center> <img src=./../src/imgs/yolopipeline.png> </center>

<br>
</br>


В базовом варианте пайплайна для PASCAL VOC использовались параметры S=7, B=2, C=20 что приводит к размеру выходного тензора: (S, S, Bx5 + C) = (7, 7, 2x5 + 20) = (7, 7, 30)

----

### Дизайн Архитектуры:

----
<br>
</br>

<center> <img src=./../src/imgs/yoloarch.png> </center>

<br>
</br>

В качестве модели YOLO использует CNN состоящую из 24 сверточных слоев и 2 полносвязных слоя. Архитектура сети основана на GoogleNet, где inception модуль заменен на чередующиеся слои 1x1 сверток (для понижения размерности в глубину карт активаций) с последующим 3х3 свертками. Вход сети имеет размер 448х448, а выход представлен в виде тензора 7х7х30.

Кроме того, есть еще более быстрая версия YOLO, которая состоит из 9 сверточных блоков вместо 24, использует ResNet и имеет меньшую глубину сверток.

В качестве функции активации используется __leaky ReLU__ и во время обучения используется __dropout__ и аугментация данных (сдвиги, масштабирование до 20%, изменения цвета, насыщенности в пространстве HSV).


----

### Loss Function - "ответственный" предиктор чей бокс имеет максимальный IoU с ground truth:

----

YOLO предсказывает несколько боксов для каждой ячейки, т.е. имеет несколько предикторов для ячейки, но во время обучения, только один предиктор отвечает за детект каждого объекта (и соответственно вносит вклад в loss). Для этого, "ответственным" назначается тот предиктор, чей бокс имеет максимальный IoU с ground truth. Это приводит к специализации этих предикторов: каждый предиктор обучается лучше предсказывать определенные размеры, пропорции и классы объектов, улучшая тем самым итоговую точность детекции.

__YOLO исопльзует Sum of Squared Errors (SSE)__ в качестве loss function. Полная функция потерь состоит из:

1. Classification Loss.
2. Localization Loss.
3. Confidence Loss.


Пройдемся по всем:


__Classification Loss:__

Согласно названию отвечает за классификацию задетекченного объекта:

#### $\sum_{i=0}^{s^2} 1_i^{obj} \sum_{c in classes} (p_i(c) - \^{p_i}(c))^2$



* $1_i^{obj}$ - равен 1, если объект присутствует в ячейке i иначе 0
* $\^{p_i}(c)$ - вероятность того, что объект класса с присутствует в ячейке i


__Localization Loss:__

<br>
</br>

<center> <img src=./../src/imgs/localization_loss.png> </center>

<br>
</br>


Измеряет ошибку предсказания координат.


* $1_i^{obj}$ - равен 1, если предиктор j в ячейке i "ответственен" За детекцию этого объекта иначе 0
* $\lambda_{coord}$ - нормировочный коэффициент (равен 5)


__Confidence Loss:__

Измеряет ошибку "объектности", то есть то что в предсказанном боксе действительно находится искомый объект:


<br>
</br>

<center> <img src=./../src/imgs/confidence_loss.png> </center>

<br>
</br>


__Full Loss:__

<br>
</br>

<center> <img src=./../src/imgs/fullLoss.png> </center>

<br>
</br>



### Обучение YOLO происходит в 2 этапа:

1. Сеть предобучается на задаче классификации (ImageNet) со входом 224х224.
2. Затем полносвязные слои заменяются сверточными, вход увеличивается до 448х448 и сеть дообучивают на задаче детекции (VOC или COCO).

YOLO может выдавать несколько детекций для одного и того же объекта. Поэтому к итоговым детекциям применяется __Non-maximum suppression (NMS) что дает прирост на 2-3% в среднем к mAP:

<br>
</br>

<center> <img src=./../src/imgs/nms.png> </center>

<br>
</br>


Минусы YOLO:

Из-за своей архитектуры (grid cell) YOLO имеет ограничения на детекцию близких друг к другу небольших объектов (например толпа лиц, стая животных и тд).


Преимущества:
1. Очень быстрая детекция, так как задача решается за один проход через сеть, и не используется sliding window или region proposals.
2. Модель может быть обучена end-to-end.
3. Используется контекст объектов, так как сеть обрабатывает изображение целиком.
4. Хорошее обобщение на объекты из других доменов (мультики, растения, объекты которые мало встречались и тд).
5. Дает меньш false positive срабатываний.


____

# YOLOv2 (YOLOv2/YOLO9000) - Better, Faster, Stronger:

----

Первая версия - недостаток - более низкая точность локализации объектов по сравнению с Fast/Faster R-CNN а также более низкий recall по сравнению с region proposals подходами. Поэтому было решено сфоркусироваться на улучшении recall и точности локализации при сохранении хорошей точности классификации и высокой скорости детекции.

Пройдемся по улучшениям:

____

### 1. Batch Normalization:

____

<br>
</br>

<center> <img src=./../src/imgs/BN.png> </center>

<br>
</br>

Добавили Batch Normalization во все сверточные слои, что усилило регуляризацию сети, и устранило необходимость в использовании dropouts что привело к увеличению mAP на 2%


____

### 2. High Resolution Classifier:

____

Первая версия YOLO:

* Обучаем на Imagenet со входом 224х224
* Увеличиваем вход до 448x448 и дотюниваем на детекцию на VOC или COCO


YOLOv2:

* Сеть обучается на задаче классификации (ImageNet) со входом 224х224
* Затем вход увеличивается до 448х448 и сеть обучается еще несколько эпох
* После сеть дотюнивается на задаче детекции (VOC/COCO)


Плюс еще увеличили mAP на 4%


____

### 3. Convolutional with Anchor Boxes:

____


<br>
</br>

<center> <img src=./../src/imgs/CNNcan.png> </center>

<br>
</br>

На ранних эпохах обучения YOLO страдает от нестабильности градиента, так как пытается предсказывать боксы произвольной формы. Эти предсказания могут быть точными для одних объектов и неточными для других, что приводит к изменениям в градиенте. Кроме того, на ранних этапах обучения предикторы соревнуются друг с другом за специализацию на определенных формах боксов. В реальных ситуациях объекты не так разнообразны: машины похожи, пешеходы имеют примерно одинаковую форму и соотношение сторон, поэтому обучение будет нестабильным, если модель будет использовать несколько наиболее общих форм боксов.

Что в итоге сделали:

Позаимствовали идею из Faster-R-CNN и начали использовать якорные боксы (anchor boxes). Вместо предсказания боксов произвольных форм YOLO предсказывает смещения для этих якорных боксов. Если ограничить диапазоны этих смещений то можно сохранить разннобразие форм и заставить каждый якорный бокс специализироваться на конкретной форме, что в итоге приведет к более стабильному градиенту модели на разных этапах.

Для этих целей, были измененые следующие части архитектуры:
1. Удалили все полносвязные слои ответственные за предсказание боксов.
2. Предсказание класса объекта вносится с уровня ячейки, и предсказывается на уровне бокса. Каждое предсказание включает в себя: 4 координаты бокса, оценку уверенности и 20 вероятностей для классов объекта (VOC). Таким образом каждая ячейка предсказывает 5х(4+1+20) = 125 значений, а выходной тензор будет иметь форму 7х7х125
3. Размер входа сети изменяется с 448х448 на 416х416 для того, чтобы иметь нечетное число ячеек (7х7 вместо 8х8) так как центр изображения содержит чаще всего большой объект, нечетное количество ячеек помогает упростить его детекцию.
4. Выходной размер сети увеличивется с 7х7 на 13х13 путем удаления одного pooling слоя.


__Как выбрать якорные боксы (anchor boxes)__:

Боксы выбираются при помощи кластеризации. Запускается K-means и центры кластеров формируют кластерные боксы.

<br>
</br>

<center> <img src=./../src/imgs/clusters.png> </center>

<br>
</br>


Подрбоная идея: Для того, чтобы найти топ-5 якорных боксов, которые лучше всего покрывают датасет, используется K-Means кластеризация и находятся центроиды для топ-k кластеров (5). Поскольку кластеризация идет на боксах а не на точках, обычное евклидово расстояние не очень подходит для вычисления расстояния между боксами и центрами кластеров. Поэтому используется метрика

### $IoU_{d(box centroid) = 1 - IoU_{box_centroid}}$

С повышением количества кластеров (якорных боксов) точность растет, однако растет и вычислительная сложность детектора.


____

### 4. Direct Location Prediction:

____

Проблема с которой столкнулись при использовании якорных точек - нестабильность модели на начальных итерациях обучения (в основном из-за предсказания (x,y) координаты бокса.  Если вспомним подход Regional Proposal Network координаты вычислялись как:


<br>
</br>

<center> <img src=./../src/imgs/drp.png> </center>

<br>
</br>

D YOLOv2 подход основан на предсказании пятерки (tx, ty, tw, th, to) и применении сигмоид для ограничения диапазона значений.


____

### 5. Fine Grained Features:

____

<br>
</br>

<center> <img src=./../src/imgs/fgf.png> </center>

<br>
</br>

Passthrough слой конкатенирует признаки из более ранних слоев с выходных слоем: 26х26х512 --> 13х13х2048 || 13х13х1024 = 13х13х3072

____

### 6. Multi-Scale Training:

____

После удаления всех полносвязных слоев сеть YOLOv2 становится Fully Convolutional Network (FCN) поэтому может работать на различных входных разрешениях.



<br>
</br>

<center> <img src=./../src/imgs/SOTAOD.png> </center>

<br>
</br>



# Часть 2:

____

# Darknet:

----

В YOLOv2 использовалась архитектура Darknet-19 которая имеет 5.5 млрд параметров (против 8.5 у первой версии). Точность достигает 92% на ImageNet. Архитектура для задач классификации Darknet-19 выглядит следующим образом:

<br>
</br>

<center> <img src=./../src/imgs/darknet.png> </center>

<center> <img src=./../src/imgs/darknet_19.png> </center>

<br>
</br>


__Обучение__:

1. Сеть предобучается на ImageNet-1000 на разрешении 224х224 (160 эпох)
2. Затем вход увеличивается до 448х448 и сеть обучается еще 10 эпох
3. Полносвязные слои удаляются, добавляется Passthrough слой и модель дотюнивается на задаче детекции 160 эпох (на COCO/VOC/OpenImages)


Датасеты для детекции объектов имеют гораздо меньше классов чем датасеты для классификации. Чтобы увеличить число классов которые YOLOv2 может детектить начали использовать во время обучения оба типа датасетов. Когда встречается изображение из классификации, то градиент идет только через loss классификатора, а в качестве "ответственного" бокса выбирается тот, который выдал больший class probability. Используется несколько softmax.

Возникает проблема: Как слить метки классов (labels) из разных датасетов? Для этого все классы объединяют в иерархическую структуру __WordTree__:


<br>
</br>

<center> <img src=./../src/imgs/wordtree.png> </center>

<br>
</br>

____

# YOLO9000:

----

Простое расширение модели YOLOv2 на 9000 классов.

Используют датасеты:
1. MS COCO (200 классов)
2. TOP-9000 классов из ImageNet (44 класса пересекаются с COCO).


YOLO9000 выдает mAP равный 19.7 для всех классов и mAP равный 16.0 для 156 классов которых нет в COCO.


____

# YOLOv3/v4/v5:

----

YOLOv3/v4 решает задачу Multi-LAbel classification и использует logistic classifiers с binary cross entropy loss.


__Softmax vs LogReg__:

<br>
</br>

<center> <img src=./../src/imgs/softmaxlogreg.png> </center>

<br>
</br>

__Bounding Box Prediction__:
<br>
</br>

<center> <img src=./../src/imgs/yolov3.png> </center>

<br>
</br>

__Predictions Across Scales__:
<br>
</br>

<center> <img src=./../src/imgs/yolov3_1.png> </center>
<center> <img src=./../src/imgs/yolov3_3.png> </center>

<br>
</br>


__Darknet-53 / DarkNet-17__
<br>
</br>

<center> <img src=./../src/imgs/darknet_53_17.png> </center>

<br>
</br>



# [https://www.youtube.com/watch?v=MPU2HistivI](https://www.youtube.com/watch?v=MPU2HistivI)


In [1]:
import os
import glob as glob
import matplotlib.pyplot as plt
import cv2
import requests

In [2]:
TRAIN = True
# Number of epochs to train for.
EPOCHS = 25

In [3]:
def download_file(url, save_name):
    url = url
    if not os.path.exists(save_name):
        file = requests.get(url)
        open(save_name, 'wb').write(file.content)
    else:
        print('File already exists')

In [4]:
if not os.path.exists('/train'):
    !curl -L "https://public.roboflow.com/ds/xKLV14HbTF?key=aJzo7msVta" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

    dirs = ['train', 'valid', 'test']

    for i, dir_name in enumerate(dirs):
        all_image_names = sorted(os.listdir(f"{dir_name}/images/"))
        for j, image_name in enumerate(all_image_names):
            if (j % 2) == 0:
                file_name = image_name.split('.jpg')[0]
                os.remove(f"{dir_name}/images/{image_name}")
                os.remove(f"{dir_name}/labels/{file_name}.txt")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0   893    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100   893  100   893    0     0    893      0  0:00:01  0:00:01 --:--:--   657

  0 38.4M    0  3262    0     0   1631      0  6:51:50  0:00:02  6:51:48  1631
  6 38.4M    6 2560k    0     0   853k      0  0:00:46  0:00:03  0:00:43 3478k
 92 38.4M   92 35.4M    0     0  9069k      0  0:00:04  0:00:04 --:--:-- 20.0M
100 38.4M  100 38.4M    0     0  9839k      0  0:00:04  0:00:04 --:--:-- 20.8M
curl: (6) Could not resolve host: ;

  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0curl: (6) Could not resolve host: unzip

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: roboflow.zip;

  0 

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'train/images/'